<a href="https://colab.research.google.com/github/anuga-community/anuga-clinic/blob/master/notebooks/notebook4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CSDMS ANUGA Clinic 2018

## Notebook 4: Setting up a Pumping Operator separated by a River Wall

Here we go through the process of creating an operator (fractional step operator) which implements a simple pump, and separate the inlet from the outlet with a River wall.  

### Installation

These notebooks have been designed to run in the google `colaboratory` environment, which provides a jupyter notebook environment running on a virtual machine on the cloud. To use this environment you need a google account so that your notebook can be saved on google drive. 

To start interacting with the notebook follow the 
`View in Colaboratory` link above. 

## Setup Environment

If on github, first follow the link `View in Colaboratory' to start running on google's colab environment. Then ....

Run the following cell to install the dependencies and some extra code for visualising on Colaboratory.

Wait until you see the comment *(4) Ready to go* before proceeding to subsequent commands. 

The install should take less than a minute (and quicker if you have already run this earlier).

In [ ]:
try:
  # On colab we can install all the packages we need via the notebook
  #
  # First download the clinic repository
  import os
  os.chdir('/content')
  !git clone https://github.com/anuga-community/anuga-clinic.git

  # Now install environment using tool
  !/bin/bash /content/anuga-clinic/anuga_tools/install_anuga_colab.sh
 
except:
  pass

# Make inline animate code available
if not 'workbookDir' in globals():
    workbookDir = os.getcwd()

import sys
sys.path.append(os.path.join(workbookDir,"anuga-clinic"))

### Import ANUGA and Tools

Once `anuga_clinic` and `anuga` have been installed we can now `import` our standard libraries.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Allow inline jshtml animations
from matplotlib import rc
rc('animation', html='jshtml')

import anuga

## The Code 

This code is taken from `anuga_core/examples/structures/run_pump.py`.


### Parameters to Setup Domain



In [ ]:
boundaryPolygon = [ [0., 0.], [0., 100.], [100.0, 100.0], [100.0, 0.0]]
wallLoc = 50.
# The boundary polygon + riverwall breaks the mesh into multiple regions
# Must define the resolution in these areas with an xy point + maximum area
# Otherwise triangle.c gets confused
length = 2.0
res = length*length*0.5
regionPtAreas = [ [99., 99., res],
                  [1., 1., res],
                  [45, 50, res],
                  [55, 50, res]]

wallHeight=10.
InitialOceanStage=2.
InitialLandStage=6.


### RiverWall Setup and Utility Procedure

We will be setting up a `riverwall` which implicitly has an associated polyline. Setup a procedure to pull out this polyline so that it can be used in the construction of the `domain`.



In [ ]:
riverWall = { 'centralWall':
                           [ [wallLoc, 0.0, wallHeight],
                             [wallLoc, 100.0, wallHeight]] 
                        }

riverWall_Par = {'centralWall':{'Qfactor':1.0}}

def toPolyline(riverWall):
  return [np.array(list(riverWall.values())[0])[:,0:2].tolist()]

### Setup Pump geometry

First setup the inlet and outlet polygonal regions of the pump. 

In [ ]:
end_point0 = [49.0,50.0]
end_point1 = [51.0,50.0]

end_points = [end_point0, end_point1]

inlet1_poly = [[[end_point0[0]-10, 45.0],[end_point0[0]-10,55],
                [end_point0[0],55],[end_point0[0],45],[end_point0[0]-10, 45.0]]]
              
inlet2_poly = [[[end_point1[0], 45.0],[end_point1[0],55],
                [end_point1[0]+10,55],[end_point1[0]+10,45],[end_point1[0], 45.0]]]
              

### Create and Setup Domain

Create  a domain with a breakline down the middle (where the riverWall will occur) and two polygonal regions where the inlet and oulet polygons will sit. 

In [ ]:
domain = anuga.create_domain_from_regions(boundaryPolygon, 
                           boundary_tags={'left': [0],
                                        'top': [1],
                                        'right': [2],
                                        'bottom': [3]},
                           maximum_triangle_area = 10.0,
                           minimum_triangle_angle = 28.0,
                           interior_regions =[ ], #[ [higherResPolygon, 1.*1.*0.5],
                                                  #  [midResPolygon, 3.0*3.0*0.5]],
                           breaklines=toPolyline(riverWall)+inlet1_poly+inlet2_poly,
                           regionPtArea=regionPtAreas,
                           use_cache=False,
                           verbose=False
                           )


#domain.set_flow_algorithm('DE1')
domain.set_name('run_pump')
domain.set_store_vertices_uniquely(True)

d_plotter = anuga.Domain_plotter(domain)

### Setup Initial Conditions

We will have a slight slope down towards the right, with water height `InitialOceanStage` on the  right, and `InitialLandStage` on the left.

In [ ]:
def topography(x,y):
    return -x/150. 

def stagefun(x,y):
    stg = InitialOceanStage*(x>=wallLoc) + InitialLandStage*(x<wallLoc)
    return stg 


# NOTE: Setting quantities at centroids is important for exactness of tests
domain.set_quantity('elevation',topography,location='centroids')     
domain.set_quantity('stage', stagefun,location='centroids')    

### Setup RiverWall 

From the `riverWall` polyline and elevation data create a riverWall  `operator`. Looking back at the definition of `riverWall` we see that this will create a wall of height `wallHeight` down the middle of the domain. 

In [ ]:
domain.riverwallData.create_riverwalls(riverWall,riverWall_Par,verbose=False) 

### Boundary conditions

Simple reflective BC all around


In [ ]:
Br = anuga.Reflective_boundary(domain)
domain.set_boundary({'left': Br, 'right': Br, 'top': Br, 'bottom':Br})

### Setup pump Operator

(1) First setup the pump characteristics

(2) Then locate and create the pump using the operator

In [ ]:
pump_function = anuga.pumping_station_function(
            domain=domain,
            pump_capacity=100.0,
            hw_to_start_pumping=0.0,
            hw_to_stop_pumping=-1.0,
            initial_pump_rate=100.0, 
            pump_rate_of_increase = 50.0, 
            pump_rate_of_decrease = 50.0, 
            verbose=True)


end_points = [[49.0,50.0],[51.0,50.0]]
pump = anuga.Internal_boundary_operator(domain, pump_function,
                                        width = 10.0,
                                        height = 1.0,
                                        apron = 10.0,
                                        end_points=end_points,
                                        verbose=True)


### Evolve

Monitor the amount of water on each side of the wall. The sum should remain constant, and the change should be match the pump capacity.

In [ ]:
region1 = anuga.Region(domain, polygon=[[0.0,0.0], [50.0,0.0], [50.0, 100.0], [0.0,100.0]])
region2 = anuga.Region(domain, polygon=[[50.0,0.0], [100.0,0.0], [100.0, 100.0], [50.0,100.0]])

for t in domain.evolve(yieldstep=2, duration=120):
    domain.print_timestepping_statistics()
    stage = domain.get_quantity('stage')
    elev  = domain.get_quantity('elevation')
    height = stage - elev
    
    #d_plotter.plot_depth_frame(vmin=2.0, vmax=6.0)

    print anuga.indent + 'Integral1 = ', height.get_integral(region=region1)
    print anuga.indent + 'Integral2 = ', height.get_integral(region=region2)
    print anuga.indent + 'Total Integral = ', height.get_integral()
    #pump.print_timestepping_statistics()

## Visualise Flow

Let's open up the `swwfile` and look at the evolution of the `depth` the `stage` and the `elev`.

In [ ]:
# Create a wrapper for contents of sww file
swwfile = 'run_pump.sww'
splotter = anuga.SWW_plotter(swwfile);

plt.triplot(splotter.triang, linewidth = 0.4);

In [ ]:
# plot stage
splotter.triang.set_mask(None)
for i,time in enumerate(splotter.time):
  print time
  splotter.save_stage_frame(frame=i, vmin=1.0, vmax=7.0)
  
splotter.make_stage_animation()

## Animation

We ca use the `FuncAnimation` function to animate a sequence of timeslices (in this case  `stage`)




In [ ]:
def update_frame(i):
  
  ax.clear()
  j = step*i
  artist = ax.tripcolor(splotter.triang, 
              facecolors = splotter.stage[j], 
              edgecolors='k', 
              cmap='Spectral',
              vmax = 6.0,
              vmin = 2.0) 
  ax.set_title('Stage (time %05.2f)' % splotter.time[j],loc='right')
  fig.colorbar(artist, cax=cax)
  

from matplotlib import animation
  
fig = plt.figure();
ax = plt.gca();
cax = fig.add_axes([0.91, 0.125, 0.03, 0.75])

# increase step to skip frames; increase interval to increase movie speed
step = 4
anim = animation.FuncAnimation(fig, update_frame, frames=len(splotter.depth)/step, interval=200);


plt.close()

anim


###  3D Surface Plots

We can use the mpl module to produce some 3D plots. We need to create vertex values given the centroid values.

In [ ]:

X = np.vstack((splotter.xc, splotter.yc)).transpose()
E = splotter.elev

splotter.nodes = np.vstack((splotter.x,splotter.y)).transpose()
import scipy

stage_v = []
for i,S in enumerate(splotter.stage):
  #print i, S.shape, X.shape
  Interp = scipy.interpolate.NearestNDInterpolator(X,S)
  stage_v.append(Interp(splotter.nodes))
  
stage_v = np.array(stage_v)

Interp = scipy.interpolate.NearestNDInterpolator(X,E)
elev_v = Interp(splotter.nodes)





from matplotlib.animation import FuncAnimation
from mpl_toolkits.mplot3d import Axes3D


fig = plt.figure();
#fig, ax = plt.subplots()
ax = plt.gca(projection='3d');

def update_image(i):
  ax.clear()
  ax.plot_trisurf(splotter.triang, elev_v, color='navajowhite', edgecolors='none', antialiased=False, shade=True);
  ax.plot_trisurf(splotter.triang, stage_v[i*step], color='dodgerblue', edgecolors='none', alpha=1.0, linewidth=0, antialiased=False, shade=True);
  ax.set_zlim(elev_v.min(), np.max([stage_v.max(), elev_v.max()]));
    

# increase step to skip frames; increase interval to increase movie speed
step = 4
anim = FuncAnimation(fig, update_image, frames=len(splotter.depth)/step, interval=200);

plt.close()

anim